## 🔰PyTorchでニューラルネットワーク基礎 #21【文章分類・Conv1d】

### 内容
* Qiitaの記事と連動しています
* Conv1dを利用したテキスト分類
* 利用するデータ：data/data_90.zip
* data_90.zipは解凍して利用

### データについて
* webサイトへの「ログイン、登録、解約」の３種類の問い合わせを分類する60個データ
* 事前にmecabなどで形態素解析をして単語にIDを割り振り、等長の長さに変換してある
* <bos>文章<eos><pad>の形で長さが11になるまで<pad>を挿入
* カテゴリー数：３
* 単語数：146
* 系列長（文章の長さ）：11

**textCNNの基本実装**
* Yoon Kim (2014) “Convolutional Neural Networks for Sentence Classification”

* 埋め込み層→Conv1D (kernel=3,4,5)→concat→FC

**【検証精度】**
* 3種類の畳み込み : 82.6% 
* 2種類の畳み込み : 78%

**【sample_20.ipynbとの比較】**
* data_60でconv2dは学習データでの精度は100%だが検証精度は50%
* data_90とデータ数を増加する。conv2dの同じ構造でも精度は70%へ上昇。データ数が重要
* data_90で構造を変更 textCNN (conv1d)を使う。2層で78%、3層で82%まで上昇。モデルの構造も重要

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
#デバイスの選択
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("利用デバイス:", device)

# 精度を計算する関数
def accuracy(y, t):
    _, argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy

利用デバイス: cuda


In [ ]:
# データの読み込み データの保存先ディレクトリは適宜変更してください。
# x: IDベクトル
# t: ラベル
x = np.loadtxt("data_90/x_id_vector.csv", delimiter=",")
t = np.loadtxt("data_90/y_labels.csv", delimiter=",")
x = torch.LongTensor(x)
t = torch.LongTensor(t)

#--------------------------------------------------------------------------------
# 初期設定
WORDS = 146  # 単語数
SEQ_LEN = 11 # x.shape[1]  # 入力するIDベクトルの長さ
CLASSES = 3  # 分類数

In [4]:
x, x_test, t, t_test = train_test_split(x,t, stratify=t,  random_state=55)

x = x.to(device)
t = t.to(device)
x_test = x_test.to(device)
t_test = t_test.to(device)
x.shape, x_test.shape

(torch.Size([67, 11]), torch.Size([23, 11]))

### CNN1Dブロックの作成
* Conv1d・正規化・プーリングを組み合わせブロックを作成する

In [5]:
class CNN1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding="same"):
        super().__init__()
        # １次元畳み込み層 -> プーリング層 -> (バッチサイズ, 特徴量ベクトル)
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn   = nn.BatchNorm1d(out_channels)
        self.act  = nn.ReLU()
        # プーリング・Dropout・全結合など追加すると本格的
        self.pool = nn.AdaptiveMaxPool1d(1)

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h)                     
        h = self.act(h)
        y = self.pool(h).squeeze(-1)        # (B, 特徴量ベクトル)
        return y

### ネットワーク全体
* 文章のIDのベクトル→埋め込み層→CNN1Dブロック→特徴量を結合→全結合層→分類へ

In [6]:
class DNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.embed = nn.Embedding(num_embeddings=WORDS, embedding_dim=10)
        self.cnn3  = CNN1D(in_channels=10, out_channels=16, kernel_size=3, padding="same")
        self.cnn4  = CNN1D(in_channels=10, out_channels=16, kernel_size=4, padding="same")
        self.cnn5  = CNN1D(in_channels=10, out_channels=16, kernel_size=5, padding="same")
        self.fc = nn.Linear(16 * 3, 3) # in_features = 特徴量×3 : ３は畳み込み層の数

    def forward(self, x):                     # x: (B, L)　B:バッチサイズ, L:系列長, E:埋め込み次元
        h = self.embed(x)                     # (B, L, E)
        # (1) 【重要】転置して、横に単語が時系列順に並ぶ形にする
        h = h.permute(0, 2, 1)                # (B, E, L) 転置 conv1dの入力
        # (2) カーネルサイズで分岐
        h3 = self.cnn3(h)   # kernel_size=3 で特徴量抽出
        h4 = self.cnn3(h)   # kernel_size=4 で特徴量抽出
        h5 = self.cnn3(h)   # kernel_size=5 で特徴量抽出

       # (3) 特徴量を結合
        v = torch.cat([h3, h4, h5], dim=1)        # (B, 3×特徴量ベクトル)
        y = self.fc(v)
        return y

In [7]:
model = DNN()
model.to(device)

DNN(
  (embed): Embedding(146, 10)
  (cnn3): CNN1D(
    (conv): Conv1d(10, 16, kernel_size=(3,), stride=(1,), padding=same)
    (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (pool): AdaptiveMaxPool1d(output_size=1)
  )
  (cnn4): CNN1D(
    (conv): Conv1d(10, 16, kernel_size=(4,), stride=(1,), padding=same)
    (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (pool): AdaptiveMaxPool1d(output_size=1)
  )
  (cnn5): CNN1D(
    (conv): Conv1d(10, 16, kernel_size=(5,), stride=(1,), padding=same)
    (bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU()
    (pool): AdaptiveMaxPool1d(output_size=1)
  )
  (fc): Linear(in_features=48, out_features=3, bias=True)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.003)

In [9]:
LOOP = 50

for epoch in range(LOOP):
    optimizer.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    acc  = accuracy(y,t)
    loss.backward()
    optimizer.step()    
    print(f"{epoch}:\tloss:{loss.item():.3f}\tacc:{acc:.3f}")

0:	loss:1.341	acc:0.328
1:	loss:1.207	acc:0.358
2:	loss:1.116	acc:0.418
3:	loss:1.073	acc:0.448
4:	loss:1.064	acc:0.448
5:	loss:1.058	acc:0.403
6:	loss:1.041	acc:0.388
7:	loss:1.013	acc:0.433
8:	loss:0.980	acc:0.507
9:	loss:0.948	acc:0.627
10:	loss:0.921	acc:0.627
11:	loss:0.898	acc:0.597
12:	loss:0.880	acc:0.657
13:	loss:0.863	acc:0.672
14:	loss:0.846	acc:0.672
15:	loss:0.826	acc:0.687
16:	loss:0.803	acc:0.687
17:	loss:0.777	acc:0.731
18:	loss:0.749	acc:0.791
19:	loss:0.722	acc:0.851
20:	loss:0.695	acc:0.881
21:	loss:0.670	acc:0.881
22:	loss:0.648	acc:0.910
23:	loss:0.626	acc:0.896
24:	loss:0.603	acc:0.896
25:	loss:0.580	acc:0.881
26:	loss:0.557	acc:0.881
27:	loss:0.534	acc:0.910
28:	loss:0.510	acc:0.925
29:	loss:0.488	acc:0.925
30:	loss:0.468	acc:0.925
31:	loss:0.449	acc:0.940
32:	loss:0.431	acc:0.940
33:	loss:0.413	acc:0.940
34:	loss:0.395	acc:0.940
35:	loss:0.378	acc:0.940
36:	loss:0.360	acc:0.955
37:	loss:0.343	acc:0.955
38:	loss:0.326	acc:0.955
39:	loss:0.311	acc:0.955
40:	loss:0

* 検証精度は目安です。
* 70%程度のこともあります。
* 86%はたまたまですので注意してください。

In [11]:
model.eval()
y_test = model(x_test)
acc = accuracy(y_test, t_test)
print(f"検証精度: {acc}")

検証精度: 0.8695652173913043
